In [1]:
import os
import joblib
from pathlib import Path
from time import time
import pandas as pd
import numpy as np
from tqdm import tqdm
from model_settings import ms
from quantlib_pricers import barriers, asians

ms.find_root(Path())
ms.__dict__

modelsdir = os.path.join(ms.root,ms.trained_models)
folders = [f for f in os.listdir(modelsdir) if f.find('Legacy')==-1 and not f.startswith('.')]
folders

['2024-11-15 164145753233 cboe spx relative asian',
 '2024-11-15 164315824655 cboe spx relative barrier']

In [2]:
model_folder = folders[0]
pricename = str(model_folder[model_folder.rfind(' ')+1:])+'_price'
model_dir = os.path.join(modelsdir,model_folder)
pickle = [f for f in os.listdir(model_dir) if f.endswith('.pkl') and f.find(model_folder)!=-1][0]
model = joblib.load(os.path.join(model_dir,pickle))
est = model['model']
df = model['train_data'].iloc[:,:-4].copy().reset_index(drop=True)
df = df.drop(columns=[c for c in df.columns if c.find(pricename)!=-1])
if pricename == 'barrier_price':
    pricer = barriers.df_barrier_price

elif pricename == 'asian_price':
    pricer = asians.df_asian_option_price

pricer

<bound method asian_option_pricer.df_asian_option_price of <quantlib_pricers.asian_option_pricer.asian_option_pricer object at 0x000001E6F5017530>>

In [3]:
feature_set = model['feature_set']
feature_set

['days_to_maturity',
 'fixing_frequency',
 'past_fixings',
 'risk_free_rate',
 'dividend_rate',
 'kappa',
 'theta',
 'rho',
 'eta',
 'v0',
 'relative_spot',
 'averaging_type',
 'w']

In [4]:
def price_row(row):
    tic = time()
    features = row[1][model['feature_set']].to_frame().T
    price = est.predict(features)[0]
    MLP = time() - tic
    return {'MLP':price,'MLP_cpu':MLP}

from joblib import Parallel, delayed
results = Parallel()(delayed(price_row)(row) for row in df.iterrows())
results = pd.DataFrame(results)
df[results.columns] = results

In [5]:
prices = pd.DataFrame(pricer(df))
df[prices.columns] = prices

In [6]:
df.to_csv(f"{ms.timetag()} {pricename[:pricename.find('_')]} performance.csv")

In [7]:
file = [f for f  in os.listdir(Path()) if f.endswith('.csv')][0]

In [8]:
df = pd.read_csv(os.path.join(Path(),file)).iloc[:,1:].copy()
df.dtypes

spot_price           float64
strike_price         float64
barrier              float64
days_to_maturity       int64
updown                object
outin                 object
w                     object
barrier_type_name     object
rebate               float64
dividend_rate        float64
risk_free_rate       float64
theta                float64
kappa                float64
rho                  float64
eta                  float64
v0                   float64
calculation_date      object
date                  object
relative_spot        float64
relative_barrier     float64
relative_rebate      float64
MLP                  float64
MLP_cpu              float64
barrier_price        float64
barrier_cpu          float64
dtype: object